<a href="https://colab.research.google.com/github/vmacf/redes_neurais/blob/master/RNN/Rede_Neural_Recorrente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!git clone https://github.com/vmacf/redes_neurais

fatal: destination path 'redes_neurais' already exists and is not an empty directory.


# Rede Neural Recorrente

Tutorial baseado no site: http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-2-implementing-a-language-model-rnn-with-python-numpy-and-theano/

In [12]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Modelagem de linguagem (Language Modeling)

Nessa aula, vamos desenvolver um gerador de linguagem utilizando rede neural recorrente (RNN). 

Por exemplo, se temos uma sentença de $m$ palavras. A modelagem de linguagem realiza a previsão de ser observar uma determinada sentença, numa base de dados:

$
\begin{aligned}
P(w_1,...,w_m) = \prod_{i=1}^{m}P(w_i \mid w_1,..., w_{i-1}) 
\end{aligned}
$


Utilizando a RNN, a probabilidade de uma sentença é o produto das probabilidades de cada palavra, dadas as palavras que vieram antes dela. Assim, a probabilidade da sentença "Ele foi comprar alguns chocolates", seria a probabilidade de "chocolate" dado "Ele foi comprar alguns", multiplicado pela probabilidade de "alguns", dado "Ele foi comprar", e assim por diante.

Por que isso é útil? Por que queremos atribuir uma probabilidade de observar uma sentença?

Podemos utilizar essa modelagem pra resolver alguns problemas de processamento de linguagem natual. Por exemplo, esse modelo pode ser usado como um mecanismo de pontuação para problemas como saber qual a próxima sentença num sistema de tradução. Intuitivamente, a sentença mais provável,  é aquela que seja gramaticalmente correta. Pontuação semelhante acontece em sistemas de reconhecimento de fala. 

Também, é possível gerar novos textos. Dada uma sequência existente de palavras,  a próxima palavra é prevista pelas probabilidades de uma próxima palavra, e esse processo é reptido até formar uma frase completa.

Note que na equação acima, a probabilidade de cada palavra é condicionada a **todas** as palavras anteriores. Na prática, muitos modelos têm dificuldade em representar essas dependências de longo prazo devido a restrições computacionais ou memória. Eles são tipicamente limitados a apenas algumas das palavras anteriores, conforme discutido na aula.



## 1) Base de dados

Serão utilizados 15000 comentários do reddit,  disponíveis em [dataset available on Google's BigQuery](https://bigquery.cloud.google.com/table/fh-bigquery:reddit_comments.2015_08). 

A ideia é que os textos gerados pelo modelo sejam parecidos com os comentários de usuários do reddit.

##Pré-Processamento

#### 1.1. Tokenize

Separa o texto em tokens: A sentença "He left!" deve ter 3 tokens: "He", "left", "!". 

#### 1.2. Remove as palavras menos frequentes

Remove as palavras que aparecem pouco nos textos, por exemplo, 1 ou 2 vezes. No código, é possível limistar o tamanho do vocabulário, pois o tamanho do vocabulário influencia na velocidade de treinamento. Todas as palavras que não estiverem no vocabulário serão trocadas por `UNKNOWN_TOKEN`. 


#### 1.3. Tokens especiais de início e fim

Aprender palavras que iniciam e terminam uma sentença é importante para delimitar as sentenças. Assim, existem os tokens  `SENTENCE_START` e `SENTENCE_END` para cada frase. 

#### 1.4. Construir matrizes de dados de treinamento

A entrada para das redes neurais recorrentes são vetores, não sequências de caracteres. Então, será realizado um mapeamento entre palavras e índices, `index_to_word` e` word_to_index`. Por exemplo, a palavra "friendly" pode estar no índice 2001. Um exemplo de treinamento $x$ pode ser "[0, 179, 341, 416]", onde 0 corresponde a "SENTENCE_START". O rótulo correspondente $y$ seria `[179, 341, 416, 1]`. 

O objetivo dessa rede é prever a próxima palavra, então $y$ é apenas o vetor $x$ deslocado por uma posição, com o último elemento sendo o símbolo `SENTENCE_END`. Em outras palavras, a predição correta para a palavra `179` é `341`.



In [13]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

# Read the data and append SENTENCE_START and SENTENCE_END tokens
print "Reading CSV file..."
with open('redes_neurais/RNN/data/reddit-comments-2015-08.csv', 'rb') as f:
    reader = csv.reader(f, skipinitialspace=True)
    reader.next()
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in reader])
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print "Parsed %d sentences." % (len(sentences))
    
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique words tokens." % len(word_freq.items())

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print "Using vocabulary size %d." % vocabulary_size
print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print "\nExemplo de frase: '%s'" % sentences[0]
print "\nExemplo de frase após o pré-processamento: '%s'" % tokenized_sentences[0]

Reading CSV file...
Parsed 79170 sentences.
Found 65498 unique words tokens.
Using vocabulary size 8000.
The least frequent word in our vocabulary is 'traction' and appeared 10 times.

Exemplo de frase: 'SENTENCE_START i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END'

Exemplo de frase após o pré-processamento: '[u'SENTENCE_START', u'i', u'joined', u'a', u'new', u'league', u'this', u'year', u'and', u'they', u'have', u'different', u'scoring', u'rules', u'than', u'i', u"'m", u'used', u'to', u'.', u'SENTENCE_END']'


## 2) Cria os dados de treinamento e teste

In [0]:
# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

### 2.1 Exemplos de sentenças:

In [15]:
# Print an training data example
x_example, y_example = X_train[17], y_train[17]
print "x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example)
print "\ny:\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example)

x:
SENTENCE_START what are n't you understanding about this ? !
[0, 51, 27, 16, 10, 861, 54, 25, 34, 69]

y:
what are n't you understanding about this ? ! SENTENCE_END
[51, 27, 16, 10, 861, 54, 25, 34, 69, 1]


## 3) Construção da RNN

Rede neural desdobrada:

![](http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg)

Funções de ativação:

$
\begin{aligned}
s_t &= \tanh(Ux_t + Ws_{t-1}) \\
o_t &= \mathrm{softmax}(Vs_t)
\end{aligned}
$



Para um tamanho de vocabulário $C = 8000$ e um tamanho de camada oculta $H = 100$, teremos:

$
\begin{aligned}
x_t & \in \mathbb{R}^{8000} \\
o_t & \in \mathbb{R}^{8000} \\
s_t & \in \mathbb{R}^{100} \\
U & \in \mathbb{R}^{100 \times 8000} \\
V & \in \mathbb{R}^{8000 \times 100} \\
W & \in \mathbb{R}^{100 \times 100} \\
\end{aligned}
$

Os pesos $U,V$ e $W$ são os parâmetros que queremos aprender. 

#### 3.1) Inicialização

Inicializar no intervalo $\left[-\frac{1}{\sqrt{n}}, \frac{1}{\sqrt{n}}\right]$ onde $n$ é o número de conexões de entrada da camada anterior. 

Os parâmetros `word_dim` é o tamanho do vocabulário, e` hidden_dim` é o tamanho da  camada oculta.

In [0]:
class RNNNumpy:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
        

#### 3.2) Forward:

In [0]:
def softmax(x):
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)

def forward_propagation(self, x):
    # The total number of time steps
    T = len(x)
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, self.word_dim))
    # For each time step...
    for t in np.arange(T):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]

RNNNumpy.forward_propagation = forward_propagation


def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)

RNNNumpy.predict = predict

## 4) Função de perda

Calcula a função de perda logarítmica. 

$
\begin{aligned}
L(y,o) = - \frac{1}{N} \sum_{n \in N} o_{n} \log y_{n}
\end{aligned}
$

In [0]:
def calculate_total_loss(self, x, y):
    L = 0
    # For each sentence...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        # Add to the loss based on how off we were
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L

def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x,y)/N

RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

### 4.1) Saída pra 1000 exemplos sem treinamento:


In [19]:
np.random.seed(10)
# Train on a small subset of the data to see what happens
model = RNNNumpy(vocabulary_size)

# Limit to 1000 examples to save time
print "Expected Loss for random predictions: %f" % np.log(vocabulary_size)
print "Actual loss: %f" % model.calculate_loss(X_train[:1000], y_train[:1000])

Expected Loss for random predictions: 8.987197


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


Actual loss: 8.987430


## 5) Treinando o RNN com SGD e Backpropagation Through Time (BPTT)

O treinamento do BPTT equivale as derivadas parciais: $\frac{\partial E}{\partial U}, \frac{\partial E}{\partial V}, \frac{\partial E}{\partial W}$. 

In [0]:
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]

RNNNumpy.bptt = bptt

## 6) Implementação SGD

O SGD é implementado em duas etapas: 1. Uma função `sdg_step`, que calcula os gradientes e executa as atualizações para um lote; 2. Um loop externo que percorre o conjunto de treinamento e ajusta a taxa de aprendizado.



In [0]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW

RNNNumpy.sgd_step = numpy_sdg_step

In [0]:
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print "%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss)
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print "Setting learning rate to %f" % learning_rate
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

### 6.1)Treino com o SGD:

In [0]:
np.random.seed(10)
# Train on a small subset of the data to see what happens
model = RNNNumpy(vocabulary_size)
losses = train_with_sgd(model, X_train[:100], y_train[:100], nepoch=10, evaluate_loss_after=1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


2019-05-14 17:56:10: Loss after num_examples_seen=0 epoch=0: 8.987468
2019-05-14 17:56:24: Loss after num_examples_seen=100 epoch=1: 8.976080
2019-05-14 17:56:38: Loss after num_examples_seen=200 epoch=2: 8.959668
2019-05-14 17:56:52: Loss after num_examples_seen=300 epoch=3: 8.928843
2019-05-14 17:57:06: Loss after num_examples_seen=400 epoch=4: 8.739638
2019-05-14 17:57:20: Loss after num_examples_seen=500 epoch=5: 6.670565
2019-05-14 17:57:33: Loss after num_examples_seen=600 epoch=6: 6.212330
2019-05-14 17:57:47: Loss after num_examples_seen=700 epoch=7: 5.970069
2019-05-14 17:58:01: Loss after num_examples_seen=800 epoch=8: 5.809121


## 7) Geração de Texto

Função que gera novas sentenças:

In [0]:
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs, _ = model.forward_propagation(new_sentence)
        sampled_word = word_to_index[unknown_token]
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            samples = np.random.multinomial(1, next_word_probs[-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str

  
num_sentences = 10
senten_min_length = 5

for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print " ".join(sent)

## Exercícios

1. Treine o modelo com diferentes configurações no número de neurônios escondidos

2. Aumente a quantidade de épocas e o tamanho do arquivo de treinamento e verifique se o erro diminuiu.
